In [179]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import geopandas as gpd
import momepy
import osmnx as ox
import folium
from src.data_processing.bikepaths import get_graph_without_bikepaths
from src.data_processing.accident_edge_metrics import calculate_metric
from src import STORAGE_PATH
import matplotlib
import networkx as nx
import momepy

target_crs = 'EPSG:2180'
map_crs = 'EPSG:4326'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load graphs

In [147]:
DATA_DIRECTORY = STORAGE_PATH / 'data'

accidents = gpd.read_file(DATA_DIRECTORY / 'accidents.geojson').to_crs(target_crs)
bike_accidents = accidents.query('POJ_ROWER > 0')
trasy_rowerowe  = gpd.read_file(DATA_DIRECTORY / 'bikeroads.geojson').to_crs(target_crs)
# trasy_rowerowe = trasy_rowerowe.query('TYP != "strefa ruchu uspokojonego 20 i 30 km/h"')
trasy_rowerowe_graph = momepy.gdf_to_nx(trasy_rowerowe.to_crs(map_crs), approach='primal')

/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/momepy/utils.py:235: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [215]:
def compute_danger_metric(
    roads_without_bikeroads,
    roads,
    number_of_accidents_weight=0.7,
    continuity_weight=0.1,
    maxspeed_weight=0.1,
    betweenes_weight=0.1,
):
    maxspeeds = get_maxspeeds(roads)
    continuity = compute_continuity_metric(roads)
    betweenes = compute_betweeness(roads)

    edges = ox.graph_to_gdfs(roads_without_bikeroads, nodes=False, edges=True).reset_index()
    calculate_metric(bike_accidents, edges, crs_to_convert=target_crs, max_dist=20)
    edges = edges.set_index(['u', 'v', 'key'])
    edges['metric'] /= edges['metric'].max()
    number_of_accidents = edges['metric'].to_dict()

    return {
        idx: (
            number_of_accidents[idx] * number_of_accidents_weight +
            maxspeeds[idx] * maxspeed_weight +
            continuity.get(idx, 0) * continuity_weight +
            betweenes.get((idx[0], idx[1]), betweenes.get((idx[1], idx[0]), 0)) * betweenes_weight
        ) for idx in edges.index
    }


def compute_continuity_metric(
    roads,
):
    roads_gdf = ox.graph_to_gdfs(
        ox.get_undirected(roads),
        nodes=False, edges=True,
        node_geometry=False, fill_edge_geometry=True
    )
    continuity = momepy.COINS(roads_gdf)
    stroke_gdf = continuity.stroke_gdf()
    cont = stroke_gdf.length / stroke_gdf.length.max()
    edge_to_continuity = {(r['u'], r['v'], r['key']): cont[r[0]] for _, r in continuity.stroke_attribute().reset_index().iterrows()}
    return edge_to_continuity


def get_maxspeeds(roads):
    def sanitize(maxspeed):
        if isinstance(maxspeed, list):
            maxspeed = maxspeed[0]
        if maxspeed is None:
            maxspeed = 50

        return int(maxspeed) if isinstance(maxspeed, str) else 50

    edges = ox.graph_to_gdfs(roads, edges=True, nodes=False)

    maxspeeds = {
        (r['u'], r['v'], r['key']): sanitize(r['maxspeed']) for _, r in edges.reset_index().iterrows()
    }
    max_maxspeed = max(maxspeeds.values())
    return {k: v / max_maxspeed for k, v in maxspeeds.items()}


def compute_betweeness(roads):
    betweenness = nx.edge_betweenness_centrality(roads, k=1000)
    max_btw = max(betweenness.values())
    return {k: v / max_btw for k, v in betweenness.items()}


In [149]:
distance = 6000
simplify = True
center = (51.107883, 17.038538)

roads = ox.graph_from_point(
    center,
    dist=distance,
    simplify=simplify,
    network_type='drive',
)
roads = ox.project_graph(roads, target_crs)
roads = ox.consolidate_intersections(roads, tolerance=15, dead_ends=True).to_undirected()

/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:499: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  subcluster_centroid = node_points.loc[wcc].unary_union.centroid
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:500: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "x"] = subcluster_centroid.x
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:501: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  gdf.loc[wcc, "y"] = subcluster_centroid.y
/home/michal/miniconda3/envs/przestrzenne/lib/python3.8/site-packages/osmnx/simplification.py:503: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead

In [150]:
roads_without_bikeroads = get_graph_without_bikepaths(roads, trasy_rowerowe, min_road_to_bike_road_dist=20)

In [216]:
danger_metric = compute_danger_metric(roads_without_bikeroads, roads)
danger_metric

{(5, 649, 0): 0.7608223225788618,
 (62, 70, 0): 0.7669598642240124,
 (102, 1645, 0): 0.016512080263073464,
 (108, 3739, 0): 0.4816269725563542,
 (120, 123, 0): 0.37398043059222325,
 (149, 1409, 0): 0.05854764607292615,
 (155, 156, 0): 0.07930440945625189,
 (155, 291, 0): 0.060624413026313155,
 (155, 289, 0): 0.06510964746090701,
 (164, 815, 0): 0.19945243895045925,
 (182, 779, 1): 0.07094755129290344,
 (190, 193, 0): 0.09097084021442768,
 (191, 193, 0): 0.09081629581252496,
 (196, 197, 0): 0.07087206049915079,
 (196, 198, 0): 0.07087206049915079,
 (196, 199, 0): 0.06093123071389965,
 (215, 216, 0): 0.1071717504410692,
 (216, 218, 0): 0.06701421373506909,
 (257, 1759, 0): 0.05757693094391484,
 (257, 1760, 0): 0.05577835394946062,
 (258, 1250, 0): 0.10887905049097654,
 (258, 1253, 0): 0.20203405793141005,
 (258, 1251, 0): 0.09457313469117347,
 (259, 1059, 0): 0.6931628272707149,
 (288, 289, 0): 0.060667638612474595,
 (288, 289, 1): 0.062265841586902945,
 (288, 290, 0): 0.0615130748120280

In [218]:
roads_proj = ox.project_graph(roads, map_crs)

node_to_location = {k: (data['y'], data['x']) for k, data in roads_proj.nodes(data=True)}

folium_map = folium.Map(
    location=(51.107883, 17.038538),
    zoom_start=12,
    tiles='CartoDBPositron',
)

lines_layer = folium.FeatureGroup('All roads')
for start, end, data in roads_proj.edges(data=True):
    start_position = node_to_location[start]
    end_position = node_to_location[end]

    folium.PolyLine(
        [start_position, end_position],
        weight=2,
        color='blue',
    ).add_to(lines_layer)
lines_layer.add_to(folium_map)

def rgb2hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))


cmap = matplotlib.cm.get_cmap('Reds')
lines_layer = folium.FeatureGroup('No bike lines')
for start, end, idx in ox.project_graph(roads_without_bikeroads, map_crs).edges:
    start_position = node_to_location[start]
    end_position = node_to_location[end]

    metric = danger_metric[(start, end, idx)]

    color = rgb2hex(*cmap(metric)[:-1])
    folium.PolyLine(
        [start_position, end_position],
        weight=3,
        color=color,
    ).add_to(lines_layer)
lines_layer.add_to(folium_map)

lines_layer = folium.FeatureGroup('Bike lines')
for start, end, data in trasy_rowerowe_graph.edges(data=True):
    weight = 1
    color = '#aed581'
    x1, y1 = start
    x2, y2 = end
    folium.PolyLine(
        [(y1, x1), (y2, x2)],
        weight=weight,
        color=color,
    ).add_to(lines_layer)

lines_layer.add_to(folium_map)
folium.LayerControl().add_to(folium_map)

folium_map.save('map.html')